### Set up the right directory for importing modules

In [ ]:
import matplotlib.pyplot as plt #to remove later 
import sys
from pathlib import Path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [ ]:
<<<<<<< local
import src.CA_model as CA
import src.analysis as an
import src.utils as ut
import numpy as np
import matplotlib.pyplot as plt
import time
from importlib import reload
=======
>>>>>>> remote

In [ ]:
import src.CA_model as CA
import src.analysis as an
import time
import matplotlib.pyplot as plt
import numpy as np
import importlib

In [ ]:
# run to reload CA_model.py and analysis.py and utils.py for updated code
reload(CA)
reload(an)
reload(ut)

#### Parameter settings

In [ ]:
size = 100                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_frac=0.2                       # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=10                                # radius of neighbourhood
N_steps=200                         # number of iterations
skip=0                              # iterations to skip (equilibration period)
seed=1000

#### Generate grids for analysis

In [ ]:
grids = CA.evolve_CA(
    size=size,
    p=p,
    update_rule=update_rule,
    true_frac=true_frac,
    k=k,
    M=M,
    N_steps=N_steps,
    skip=skip,
    seed=seed,
)

#### Plot starting and final grid

In [ ]:
an.show_grids([grids[0], grids[-1]], [skip, N_steps])

#### Create GIF Animation

In [ ]:
<<<<<<< local
an.animate_grids(grids, dpi=100)
=======
size_list, fit = an.cluster_sizes(grids[100:]) #take only grids after 100 iterations
beta = fit.truncated_power_law.alpha #truncated power law to that distribution
s_char = 1 / (fit.truncated_power_law.Lambda)
R, p = fit.distribution_compare("truncated_power_law", "exponential")

print("Scaling exponent: ", beta)
print("Characteristic length: ", s_char) #typical max cluster size before exponential cutoff
print("R: ", R)
print("p?: ", p)
>>>>>>> remote

<<<<<<< local
#### Save data
=======
an.plot_cluster_size_distr(size_list=size_list, fit=fit) #Cumulative distirbution
>>>>>>> remote

In [ ]:
# if the equilibration period was not skipped yet, correct for that before saving
if skip == 0:
    skip = 100
    grids = grids[skip:]

ut.save_data(grids, size, update_rule, true_frac, k, M, N_steps, skip, seed=seed)

#### Load data

In [ ]:
loaded_grids = ut.load_data(size, update_rule, true_frac, k, M, N_steps, skip, seed=seed)

# check that the saved data is correctly recovered from the original data
assert np.all(loaded_grids == grids)

#### Plot cluster size distribution

In [ ]:
size_list, fit = an.cluster_sizes(loaded_grids)
alpha = fit.truncated_power_law.alpha
s_char = 1 / (fit.truncated_power_law.Lambda)
R, p = fit.distribution_compare("truncated_power_law", "exponential", normalized_ratio=True)

print("Scaling exponent: ", alpha)
print("Characteristic length: ", s_char)
print("Loglikelihood ratio (if positive, (truncated) power law more likely than exponential): ", R)
print("Significance value: ", p)

## Scalon rule with different weight on the local and global rule 

In [ ]:
<<<<<<< local
fig = an.plot_cluster_size_distr(size_lists=[size_list], fits=[fit])
=======
"""
Difference rules weight """

#add how it should look like on a grid 

update_rule = CA.update_Scanlon2007 # function containing update rule
phi_values = [0.0, 0.25, 0.5, 0.75, 1] #the different weight of the local rule 
results = {}

for phi in phi_values: 
    print(f"\nRunning Scalon_exp with phi = {phi}")
    all_grids = []

    for seed in range(N_evolutions): #we run 5 independent CA evolutions for each phi
        start = time.time() #function that returns the time until another point, measure performance 
        grids = CA.evolve_CA(
            size = size, 
            p = p, 
            update_rule = CA.update_Scanlon_exp, 
            true_frac=true_frac, 
            k=k,
            M = M, 
            N_steps = N_steps, #the only parameter that changes 
            skip = skip,
            seed = seed , 
            phi = phi
        )
        all_grids.append(grids)
        end = time.time()
        print(f"Evolution {seed+1}/{N_evolutions} completed in {end-start: .2f} s'")

    #Flatten all grids
    combined_grids = [g for grid_list in all_grids for g in grid_list] #this is only used to analayse the different grids but putting them next to each other? 

    #Compute cluster sizes and fit 
    size_list, fit = an.cluster_sizes(combined_grids) #size_list gives a list of all cluster sizes, and fit models the distribution 
    results[phi] = (size_list, fit)

    #Print parameters
    beta = fit.truncated_power_law.alpha 
    s_char = 1 /fit.truncated_power_law.Lambda
    R, p_val = fit.distribution_compare("truncated_power_law", "exponential") # these are just statistical test to compare the 2 model, if R>0 -> the truncated power law fits better, if p<0.05 then the difference between the fit of the 2 models is statistically significant 

    print(f" beta = {beta:.3f}") 
    print(f" s_char = {s_char:.3f}") 
    print(f" R = {R:.3f}, p = {p_val:.3f}") 
    
    # Plot 
    an.plot_cluster_size_distr(size_list, fit)

>>>>>>> remote

## Averaging over several datasets

In [ ]:
size = 200                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_frac=0.2                       # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=10                                # radius of neighbourhood
N_steps=200                         # number of iterations
skip=100                            # iterations to skip (equilibration period)

N_evolutions = 5                    # number of full evolutions to generate for this set of parameters
all_grids = []

for seed in range(N_evolutions):
    start = time.time()
    grids = CA.evolve_CA(
        size=size,
        p=p,
        update_rule=update_rule,
        true_frac=true_frac,
        k=k,
        M=M,
        N_steps=N_steps,
        skip=skip,
        seed=seed,
    )
    all_grids.append(grids)
    end = time.time()
    print(f"Grid evolution {seed+1} out of {N_evolutions} completed in {end-start} seconds.")

In [ ]:
# flatten the list of lists into a 1D list of grids
combined_grids = [grid for grid_list in all_grids for grid in grid_list]

In [ ]:
size_list, fit = an.cluster_sizes(combined_grids)
alpha = fit.truncated_power_law.alpha
s_char = 1 / (fit.truncated_power_law.Lambda)
R, p = fit.distribution_compare("truncated_power_law", "exponential", normalized_ratio=True)

print("Scaling exponent: ", alpha)
print("Characteristic length: ", s_char)
print("Loglikelihood ratio (if positive, (truncated) power law more likely than exponential): ", R)
print("Significance value: ", p)

In [ ]:
an.plot_cluster_size_distr(size_list=[size_list], fit=[fit])

## Comparing different sets of parameters

#### Generating data

In [ ]:
size = 500                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_fracs=np.arange(0.05,0.7,0.05) # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=5                                 # radius of neighbourhood
N_steps=200                         # number of iterations
skip=100                            # iterations to skip (equilibration period)

# all_grids = []

for i in range(len(true_fracs)):
    start = time.time()
    grids = CA.evolve_CA(
        size=size,
        p=p,
        update_rule=update_rule,
        true_frac=true_fracs[i],
        k=k,
        M=M,
        N_steps=N_steps,
        skip=skip,
        seed=i,
    )
    # all_grids.append(grids)
    end = time.time()
    print(f"Grid evolution {i+1} out of {len(true_fracs)} completed in {end-start} seconds.")

    ut.save_data(grids, size, update_rule, true_frac, k, M, N_steps, skip, seed)


#### Analysis

In [ ]:
# parameters for which data should be analysed
size = 500                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_fracs=np.arange(0.05,0.7,0.05) # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=5                                 # radius of neighbourhood
N_steps=200                         # number of iterations
skip=100                            # iterations to skip (equilibration period)

size_lists = []
fits = []

for i in range(len(true_fracs)):
    loaded_grids = ut.load_data(size, update_rule, np.round(true_fracs[i],2), k, M, N_steps, skip, seed)
    # retrieve the cumulative cluster size distribution
    size_list, fit = an.cluster_sizes(loaded_grids)
    size_lists.append(size_list)
    fits.append(fit)

In [ ]:
an.plot_cluster_size_distr(size_lists=size_lists, fits=fits)

In [ ]:
# parameters for which data should be analysed
size = 500                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_fracs=np.arange(0.05,0.7,0.05) # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=10                                 # radius of neighbourhood
N_steps=200                         # number of iterations
skip=100                            # iterations to skip (equilibration period)
seed=0

size_lists = []
fits = []

for i in range(len(true_fracs)):
    loaded_grids = ut.load_data(size, update_rule, np.round(true_fracs[i],2), k, M, N_steps, skip, seed)
    # retrieve the cumulative cluster size distribution
    size_list, fit = an.cluster_sizes(loaded_grids)
    size_lists.append(size_list)
    fits.append(fit)

In [ ]:
an.plot_cluster_size_distr(size_lists=size_lists, fits=fits)